### 0: Retrieve Data

In [1]:
import pandas as pd
import os
import sys

# Add the project root to sys.path (two levels up from app/notebooks)
project_root = os.path.abspath("../..")
sys.path.append(project_root)

print("Current working directory:", os.getcwd())
print("Project root added to sys.path:", project_root)

# Define the data directory at the project root
data_dir = os.path.join(project_root, "data")
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Define paths for CSV files
ndt_csv_path = os.path.join(data_dir, "ndt.csv")
returns_csv_path = os.path.join(data_dir, "returns_market_cap.csv")

# Flag to indicate if both CSV files exist
data_loaded = False

if os.path.exists(ndt_csv_path) and os.path.exists(returns_csv_path):
    print("Both CSV files exist. Loading data from CSV...")
    ndt = pd.read_csv(ndt_csv_path, low_memory=False)
    returns_and_market_cap = pd.read_csv(returns_csv_path)
    data_loaded = True
else:
    print("At least one CSV file is missing.")

if not data_loaded:
    from app.services.snowflake_query_executor import SnowflakeQueryExecutor

    # Create a Snowflake query executor
    print("Authorize Snowflake connection with Two-Factor Authentication...")
    executor = SnowflakeQueryExecutor(
        user=os.getenv('USER'),
        password=os.getenv('PASSWORD'),
        account=os.getenv('ACCOUNT'),
        warehouse=os.getenv('WAREHOUSE'),
        database=os.getenv('DATABASE'),
        schema=os.getenv('SCHEMA')
    )

    # Build SQL file paths (located inside app/sql/)
    ndt_sql_query_file_path = os.path.join(project_root, "app", "sql", "fetch_ndt.sql")
    market_cap_returns_sql_query_file_path = os.path.join(project_root, "app", "sql", "fetch_returns_market_cap.sql")

    # Retrieve the non-derivative transactions data if missing
    if not os.path.exists(ndt_csv_path):
        print("Fetching non-derivative transactions data from Snowflake...")
        ndt = executor.execute_query_from_file(ndt_sql_query_file_path)
        ndt.to_csv(ndt_csv_path, index=False)
    else:
        print("Non-derivative transactions CSV already exists.")

    # Retrieve market cap and returns data if missing
    if not os.path.exists(returns_csv_path):
        print("Fetching market cap and returns data from Snowflake...")
        returns_and_market_cap = executor.execute_query_from_file(market_cap_returns_sql_query_file_path)
        returns_and_market_cap.to_csv(returns_csv_path, index=False)
    else:
        print("Market cap and returns CSV already exists.")

Current working directory: /Users/atang/Documents/verdad/insider-transaction-research/app/notebooks
Project root added to sys.path: /Users/atang/Documents/verdad/insider-transaction-research
Both CSV files exist. Loading data from CSV...


### 1: Data Preparation and Cleaning

In [2]:
from pandas.tseries.offsets import MonthEnd

# Convert date columns to datetime for consistent handling
# MONTH_END in returns_and_market_cap is the end-of-month date for returns/market cap
returns_and_market_cap['MONTH_END'] = pd.to_datetime(returns_and_market_cap['MONTH_END'])

# TRANS_DATE in ndt is the transaction date; use errors='coerce' to handle invalid dates
ndt['TRANS_DATE'] = pd.to_datetime(ndt['TRANS_DATE'], errors='coerce')

# Create a month-end column in ndt by rounding transaction dates to the nearest month end
ndt['month_end'] = ndt['TRANS_DATE'] + MonthEnd(0)

# Check for and drop rows with invalid transaction dates (NaT)
invalid_dates = ndt['TRANS_DATE'].isna().sum()
if invalid_dates > 0:
    print(f"Warning: Dropped {invalid_dates} rows with invalid TRANS_DATE.")
    ndt = ndt.dropna(subset=['TRANS_DATE'])

# Filter ndt to include only buy ('P') and sell ('S') transactions
trades = ndt[ndt['TRANS_CODE'].isin(['P', 'S'])].copy()

# Collapse duplicate rows so each ACCESSION_NUMBER is one trade
trades = trades.groupby('ACCESSION_NUMBER', as_index=False).agg({
    'TRANS_DATE': 'first',
    'TRANS_CODE': 'first',
    'EQUITY_SWAP_INVOLVED': 'first',
    'TRANS_SHARES': 'sum',
    'TRANS_PRICEPERSHARE': 'first',
    'TRANS_ACQUIRED_DISP_CD': 'first',
    'SHRS_OWND_FOLWNG_TRANS': 'first',
    'DIRECT_INDIRECT_OWNERSHIP': 'first',
    'COMPANY_ID': 'first',
    'RPTOWNERNAME': 'first',
    'RPTOWNER_RELATIONSHIP': 'first',
    'RPTOWNER_TITLE': 'first',
    'month_end': 'first'
})

# Calculate the dollar value of each trade (shares * price per share)
trades['trade_value'] = trades['TRANS_SHARES'] * trades['TRANS_PRICEPERSHARE']

print("Sample of trades data after preparation:")
print(trades.head().to_markdown(index=False))

Sample of trades data after preparation:
| ACCESSION_NUMBER     | TRANS_DATE          | TRANS_CODE   |   EQUITY_SWAP_INVOLVED |   TRANS_SHARES |   TRANS_PRICEPERSHARE | TRANS_ACQUIRED_DISP_CD   |   SHRS_OWND_FOLWNG_TRANS | DIRECT_INDIRECT_OWNERSHIP   |   COMPANY_ID | RPTOWNERNAME   | RPTOWNER_RELATIONSHIP   | RPTOWNER_TITLE                 | month_end           |      trade_value |
|:---------------------|:--------------------|:-------------|-----------------------:|---------------:|----------------------:|:-------------------------|-------------------------:|:----------------------------|-------------:|:---------------|:------------------------|:-------------------------------|:--------------------|-----------------:|
| 0000001750-06-000002 | 2006-01-04 00:00:00 | S            |                      0 |         144360 |                 24.64 | D                        |                 18810    | D                           |       168154 | STORCH DAVID P | Director,Officer        | C

## 2: Market Returns Calculation

In [3]:
import numpy as np
import pandas as pd

# Assume returns_and_market_cap is your DataFrame with columns:
# 'MONTH_END', 'MARKET_CAP_USD', 'RETURN_LEAD_1_MONTHS', 
# 'RETURN_LEAD_3_MONTHS', 'RETURN_LEAD_12_MONTHS'

# Drop rows with missing MARKET_CAP_USD for a clean weighting process
df = returns_and_market_cap.dropna(subset=['MARKET_CAP_USD']).copy()

# Compute market-cap weighted returns per MONTH_END using np.average for each horizon
weighted_returns = df.groupby('MONTH_END').apply(
    lambda group: pd.Series({
        'mkt_1': np.average(group['RETURN_LEAD_1_MONTHS'], weights=group['MARKET_CAP_USD']),
        'mkt_3': np.average(group['RETURN_LEAD_3_MONTHS'], weights=group['MARKET_CAP_USD']),
        'mkt_12': np.average(group['RETURN_LEAD_12_MONTHS'], weights=group['MARKET_CAP_USD']),
    }),
    include_groups=False
).reset_index()

# Build a dictionary of market returns, renaming columns to match your original naming
market_returns = {}
for horizon in ['1', '3', '12']:
    col = 'mkt_' + horizon
    market_returns[f'market_return_{horizon}m'] = (
        weighted_returns[['MONTH_END', col]]
        .rename(columns={col: f'market_return_{horizon}m'})
    )

# Print sample market returns for each horizon
for horizon in ['1', '3', '12']:
    key = f'market_return_{horizon}m'
    print(f"Sample of {key} market returns:")
    print(market_returns[key].head().to_markdown(index=False))

# Merge the weighted market returns back into the main DataFrame
df = df.merge(weighted_returns, on='MONTH_END', how='left')

# Calculate excess returns: individual returns minus the market weighted return
df['excess_return_1m']  = df['RETURN_LEAD_1_MONTHS']  - df['mkt_1']
df['excess_return_3m']  = df['RETURN_LEAD_3_MONTHS']  - df['mkt_3']
df['excess_return_12m'] = df['RETURN_LEAD_12_MONTHS'] - df['mkt_12']

# Create size quartiles based on MARKET_CAP_USD
df['SIZE_QUARTILE'] = pd.qcut(df['MARKET_CAP_USD'], q=4, labels=[1, 2, 3, 4])

# Convert MONTH_END to datetime (if needed)
df['MONTH_END'] = pd.to_datetime(df['MONTH_END'], errors='coerce')

return_df_final = df

# Print a sample of the final DataFrame with excess returns
print("Sample of final DataFrame with excess returns:")
print(return_df_final.head().to_markdown(index=False))

Sample of market_return_1m market returns:
| MONTH_END           |   market_return_1m |
|:--------------------|-------------------:|
| 1962-01-31 00:00:00 |          0.0886072 |
| 1962-02-28 00:00:00 |         -0.0697672 |
| 1962-03-31 00:00:00 |          0.0249994 |
| 1962-04-30 00:00:00 |         -0.414634  |
| 1962-05-31 00:00:00 |          0.0520832 |
Sample of market_return_3m market returns:
| MONTH_END           |   market_return_3m |
|:--------------------|-------------------:|
| 1962-01-31 00:00:00 |           0.037974 |
| 1962-02-28 00:00:00 |          -0.44186  |
| 1962-03-31 00:00:00 |          -0.36875  |
| 1962-04-30 00:00:00 |          -0.256097 |
| 1962-05-31 00:00:00 |           0.375    |
Sample of market_return_12m market returns:
| MONTH_END           |   market_return_12m |
|:--------------------|--------------------:|
| 1962-01-31 00:00:00 |           -0.319621 |
| 1962-02-28 00:00:00 |           -0.331395 |
| 1962-03-31 00:00:00 |           -0.234376 |
| 1962-04-

## 3: Merging Trades with Returns Data

In [4]:
# Merge trades with returns_and_market_cap to align trade data with returns and market cap
# Use left join to keep all trades, even if no matching returns data exists
merged = trades.merge(return_df_final, 
                      left_on=['COMPANY_ID', 'month_end'], 
                      right_on=['COMPANY_ID', 'MONTH_END'], 
                      how='left')#.drop(columns='MONTH_END')  # Drop redundant MONTH_END column

print("Sample of merged data with excess returns:")
print(merged.head().to_markdown(index=False))

Sample of merged data with excess returns:
| ACCESSION_NUMBER     | TRANS_DATE          | TRANS_CODE   |   EQUITY_SWAP_INVOLVED |   TRANS_SHARES |   TRANS_PRICEPERSHARE | TRANS_ACQUIRED_DISP_CD   |   SHRS_OWND_FOLWNG_TRANS | DIRECT_INDIRECT_OWNERSHIP   |   COMPANY_ID | RPTOWNERNAME   | RPTOWNER_RELATIONSHIP   | RPTOWNER_TITLE                 | month_end           |      trade_value | MONTH_END           |   RETURN_LEAD_1_MONTHS |   RETURN_LEAD_3_MONTHS |   RETURN_LEAD_12_MONTHS |   MARKET_CAP_USD |       mkt_1 |      mkt_3 |   mkt_12 |   excess_return_1m |   excess_return_3m |   excess_return_12m |   SIZE_QUARTILE |
|:---------------------|:--------------------|:-------------|-----------------------:|---------------:|----------------------:|:-------------------------|-------------------------:|:----------------------------|-------------:|:---------------|:------------------------|:-------------------------------|:--------------------|-----------------:|:--------------------|-----------

## 4: Title Mapping and Quantile Assignment

In [5]:
import os
import pandas as pd
import string

project_root = '/Users/atang/Documents/verdad/insider-transaction-research'
title_mapping_path = os.path.join(project_root, "app", "resources", "title_mapping.csv")

title_mapper = pd.read_csv(title_mapping_path)

# Drop duplicate titles to ensure uniqueness
title_mapper_unique = title_mapper.drop_duplicates(subset='title')

mapping_dict = title_mapper_unique.set_index('title').to_dict('index')

merged['title_lower'] = merged['RPTOWNER_TITLE'].str.lower().str.strip()

def map_title(title):
    if not title:
        roles = title_mapper.columns[1:]
        return {role: 1.0 if role.upper() == 'OTHER' else 0.0 for role in roles}
    cleaned_title = title.translate(str.maketrans('', '', string.punctuation)).lower().strip()
    if cleaned_title in mapping_dict:
        return mapping_dict[cleaned_title]
    else:
        roles = title_mapper.columns[1:]
        return {role: 1.0 if role.upper() == 'OTHER' else 0.0 for role in roles}

mapped_roles = merged['title_lower'].apply(map_title).apply(pd.Series)
merged_df = pd.concat([merged, mapped_roles], axis=1)
merged_df.drop(columns=['title_lower'], inplace=True)
print(merged_df.head().to_markdown(index=False))

| ACCESSION_NUMBER     | TRANS_DATE          | TRANS_CODE   |   EQUITY_SWAP_INVOLVED |   TRANS_SHARES |   TRANS_PRICEPERSHARE | TRANS_ACQUIRED_DISP_CD   |   SHRS_OWND_FOLWNG_TRANS | DIRECT_INDIRECT_OWNERSHIP   |   COMPANY_ID | RPTOWNERNAME   | RPTOWNER_RELATIONSHIP   | RPTOWNER_TITLE                 | month_end           |      trade_value | MONTH_END           |   RETURN_LEAD_1_MONTHS |   RETURN_LEAD_3_MONTHS |   RETURN_LEAD_12_MONTHS |   MARKET_CAP_USD |       mkt_1 |      mkt_3 |   mkt_12 |   excess_return_1m |   excess_return_3m |   excess_return_12m |   SIZE_QUARTILE |   CEO |   CFO |   COO |   CTO |   CIO |   CSO |   PRESIDENT |   VP |   CHAIRMAN |   OTHER |
|:---------------------|:--------------------|:-------------|-----------------------:|---------------:|----------------------:|:-------------------------|-------------------------:|:----------------------------|-------------:|:---------------|:------------------------|:-------------------------------|:--------------------|---

In [15]:
import pandas as pd
import numpy as np
import string

# ----- Create cleaned_title from one-hot encodings -----
role_cols = ['CEO', 'CFO', 'COO', 'CTO', 'CIO', 'CSO', 'PRESIDENT', 'VP',
             'CHAIRMAN', 'OTHER', 'Director', 'Officer', 'TenPercentOwner']

def combine_roles(row):
    roles = [role for role in role_cols if row.get(role, 0) == 1]
    # For tail analysis, we simply pick the first flagged role (or empty string if none)
    return "" if not roles else roles[0]

merged_df['cleaned_title'] = merged_df.apply(combine_roles, axis=1)

# ----- Create hit flags for each horizon -----
# For purchases (TRANS_CODE "P"), a hit is when the excess return is > 0;
# for sales (TRANS_CODE "S"), a hit is when the excess return is < 0.
merged_df['hit_1m'] = np.where(
    merged_df['TRANS_CODE'] == 'P',
    merged_df['excess_return_1m'] > 0,
    np.where(merged_df['TRANS_CODE'] == 'S', merged_df['excess_return_1m'] < 0, np.nan)
)
merged_df['hit_3m'] = np.where(
    merged_df['TRANS_CODE'] == 'P',
    merged_df['excess_return_3m'] > 0,
    np.where(merged_df['TRANS_CODE'] == 'S', merged_df['excess_return_3m'] < 0, np.nan)
)
merged_df['hit_12m'] = np.where(
    merged_df['TRANS_CODE'] == 'P',
    merged_df['excess_return_12m'] > 0,
    np.where(merged_df['TRANS_CODE'] == 'S', merged_df['excess_return_12m'] < 0, np.nan)
)

# ----- Compute Monthly Averages for Each Group -----
# Group by SIZE_QUARTILE, cleaned_title, TRANS_CODE, and month_end.
# Calculate the monthly average excess returns and monthly hit rates.
monthly_group = merged_df.groupby(
    ['SIZE_QUARTILE', 'cleaned_title', 'TRANS_CODE', 'month_end']
).agg(
    avg_excess_return_1m = ('excess_return_1m', 'mean'),
    avg_excess_return_3m = ('excess_return_3m', 'mean'),
    avg_excess_return_12m = ('excess_return_12m', 'mean'),
    hit_rate_1m = ('hit_1m', 'mean'),
    hit_rate_3m = ('hit_3m', 'mean'),
    hit_rate_12m = ('hit_12m', 'mean')
).reset_index()

# ----- Group Across Months and Compute Cumulative and Annualized Returns -----
def calc_group_cum_returns(group):
    n = group['month_end'].nunique()  # number of unique months in the group
    if n == 0:
        return pd.Series({
            'n_months': 0,
            'cum_excess_1m': np.nan,
            'annualized_excess_return_1m': np.nan,
            'avg_hit_rate_1m': np.nan,
            'cum_excess_3m': np.nan,
            'annualized_excess_return_3m': np.nan,
            'avg_hit_rate_3m': np.nan,
            'cum_excess_12m': np.nan,
            'annualized_excess_return_12m': np.nan,
            'avg_hit_rate_12m': np.nan
        })
    # Calculate cumulative return for each horizon as the product over months of (1 + monthly average)
    cum1 = (1 + group['avg_excess_return_1m']).prod()
    cum3 = (1 + group['avg_excess_return_3m']).prod()
    cum12 = (1 + group['avg_excess_return_12m']).prod()
    # Annualize using the formula:
    # annualized_excess_return = log(cumulative_return) / (n * (horizon/12))
    ann1 = np.log(cum1) / (n * (1/12)) if cum1 > 0 else np.nan
    ann3 = np.log(cum3) / (n * (3/12)) if cum3 > 0 else np.nan
    ann12 = np.log(cum12) / (n * (12/12)) if cum12 > 0 else np.nan
    # Average monthly hit rates for each horizon
    avg_hit_rate_1m = group['hit_rate_1m'].mean()
    avg_hit_rate_3m = group['hit_rate_3m'].mean()
    avg_hit_rate_12m = group['hit_rate_12m'].mean()
    return pd.Series({
        'n_months': n,
        'cum_excess_1m': cum1,
        'annualized_excess_return_1m': ann1,
        'avg_hit_rate_1m': avg_hit_rate_1m,
        'cum_excess_3m': cum3,
        'annualized_excess_return_3m': ann3,
        'avg_hit_rate_3m': avg_hit_rate_3m,
        'cum_excess_12m': cum12,
        'annualized_excess_return_12m': ann12,
        'avg_hit_rate_12m': avg_hit_rate_12m
    })

# Group across all months by SIZE_QUARTILE, cleaned_title, and TRANS_CODE.
grouped_cum = monthly_group.groupby(
    ['SIZE_QUARTILE', 'cleaned_title', 'TRANS_CODE']
).apply(calc_group_cum_returns).reset_index()

/var/folders/d2/h05xc7x1181fd8jxpltdv9v40000gn/T/ipykernel_12849/1621809721.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_group = merged_df.groupby(
/var/folders/d2/h05xc7x1181fd8jxpltdv9v40000gn/T/ipykernel_12849/1621809721.py:92: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_cum = monthly_group.groupby(
/var/folders/d2/h05xc7x1181fd8jxpltdv9v40000gn/T/ipykernel_12849/1621809721.py:94: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Eithe

In [16]:

# ----- Extract Tail Performers for Each Horizon -----
# For purchases (TRANS_CODE "P"), higher annualized excess return is desirable.
purchase_groups = grouped_cum[grouped_cum['TRANS_CODE'] == 'P'].copy()
tail_buy_1m = purchase_groups.sort_values(by='annualized_excess_return_1m', ascending=False).head(10)
tail_buy_3m = purchase_groups.sort_values(by='annualized_excess_return_3m', ascending=False).head(10)
tail_buy_12m = purchase_groups.sort_values(by='annualized_excess_return_12m', ascending=False).head(10)

# For sales (TRANS_CODE "S"), more negative (lower) annualized excess return is desirable.
sale_groups = grouped_cum[grouped_cum['TRANS_CODE'] == 'S'].copy()
tail_sell_1m = sale_groups.sort_values(by='annualized_excess_return_1m', ascending=True).head(10)
tail_sell_3m = sale_groups.sort_values(by='annualized_excess_return_3m', ascending=True).head(10)
tail_sell_12m = sale_groups.sort_values(by='annualized_excess_return_12m', ascending=True).head(10)

# ----- Subset the columns for each horizon only -----
cols_1m = ['SIZE_QUARTILE', 'cleaned_title', 'TRANS_CODE', 'n_months',
           'cum_excess_1m', 'annualized_excess_return_1m', 'avg_hit_rate_1m']
cols_3m = ['SIZE_QUARTILE', 'cleaned_title', 'TRANS_CODE', 'n_months',
           'cum_excess_3m', 'annualized_excess_return_3m', 'avg_hit_rate_3m']
cols_12m = ['SIZE_QUARTILE', 'cleaned_title', 'TRANS_CODE', 'n_months',
            'cum_excess_12m', 'annualized_excess_return_12m', 'avg_hit_rate_12m']

tail_buy_1m = tail_buy_1m[cols_1m]
tail_buy_3m = tail_buy_3m[cols_3m]
tail_buy_12m = tail_buy_12m[cols_12m]

tail_sell_1m = tail_sell_1m[cols_1m]
tail_sell_3m = tail_sell_3m[cols_3m]
tail_sell_12m = tail_sell_12m[cols_12m]

print("\nTop Tail Performers for Purchases (1m):")
print(tail_buy_1m.to_markdown(index=False))
print("\nTop Tail Performers for Purchases (3m):")
print(tail_buy_3m.to_markdown(index=False))
print("\nTop Tail Performers for Purchases (12m):")
print(tail_buy_12m.to_markdown(index=False))

print("\nTop Tail Performers for Sales (1m):")
print(tail_sell_1m.to_markdown(index=False))
print("\nTop Tail Performers for Sales (3m):")
print(tail_sell_3m.to_markdown(index=False))
print("\nTop Tail Performers for Sales (12m):")
print(tail_sell_12m.to_markdown(index=False))


Top Tail Performers for Purchases (1m):
|   SIZE_QUARTILE | cleaned_title   | TRANS_CODE   |   n_months |   cum_excess_1m |   annualized_excess_return_1m |   avg_hit_rate_1m |
|----------------:|:----------------|:-------------|-----------:|----------------:|------------------------------:|------------------:|
|               2 | CFO             | P            |        306 |        46.8674  |                     0.150875  |          0.522146 |
|               1 | OTHER           | P            |        306 |        22.1504  |                     0.121484  |          0.445534 |
|               1 | PRESIDENT       | P            |        306 |        12.5704  |                     0.0992683 |          0.512053 |
|               2 | VP              | P            |        306 |        11.1985  |                     0.0947365 |          0.51426  |
|               1 | VP              | P            |        306 |         8.75376 |                     0.0850778 |          0.495761 |
|      